# Pruning model

In [ ]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.optim as optim
import torchdata
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

In [ ]:
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
  def __init__(self):
    super(LeNet,self).__init__()
    self.conv1=nn.Conv2d(3,6,5)
    self.conv2=nn.Conv2d(6,16,5)
    self.fc1=nn.Linear(16*5*5,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)
  def forward(self,x):
    x=self.conv1(x)
    x=F.relu(x)
    x=F.max_pool2d(x,(2,2))
    x=self.conv2(x)
    x=F.relu(x)
    x=F.max_pool2d(x,2)
    x=x.view(-1,int(x.nelement()/x.shape[0]))
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    y=self.fc3(x)
    return y
model=LeNet().to(device=device)

In [ ]:
#torch.manual_seed(42)

learning_rate = 0.001
batch_size = 64
num_epochs = 20

# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
train_dataset = CIFAR10(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    # Testing loop
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(test_loader):
            outputs = model(data)
            loss = criterion(outputs, targets)

            if (batch_idx + 1) % 100 == 0:
                print(f'Test Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(test_loader)}], Loss: {loss.item():.4f}')
# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/Tuto/model.pth')
# Calculate the size of the model
total_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
total_size_megabytes = total_size_bytes / (1024 * 1024)

print(f"Total size of the model: {total_size_megabytes:.2f} MB")

In [ ]:
#Test before purned
start_time = time.time()
with torch.no_grad():
    for batch_idx, (data, targets) in enumerate(test_loader):
        outputs = model(data)
        loss = criterion(outputs, targets)

        if (batch_idx + 1) % 100 == 0:
           print(f'Model  Loss: {loss.item():.4f}')
end_time = time.time()
elapsed_time1 = end_time - start_time
print(f"Elapsed time: {elapsed_time1} seconds")
## Purned model
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        #prune.l1_structured(module, name='weight', amount=0.3)
        prune.random_unstructured(module,name="weight",amount=0.1)
# Calculate the size of the model
total_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
total_size_megabytes = total_size_bytes / (1024 * 1024)
#Test aftre purned
start_time = time.time()
with torch.no_grad():
    for batch_idx, (data, targets) in enumerate(test_loader):
        outputs = model(data)
        loss = criterion(outputs, targets)

        if (batch_idx + 1) % 100 == 0:
           print(f'Model purned Loss: {loss.item():.4f}')
end_time = time.time()
elapsed_time2 = end_time - start_time
print(f"Elapsed time: {elapsed_time2} seconds")

In [ ]:
# Prune the model
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        #prune.l1_structured(module, name='weight', amount=0.3)
        prune.random_unstructured(module,name="weight",amount=0.5)

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
       prune.remove(module, 'weight')
# Calculate the size of the model
total_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())

total_size_megabytes = total_size_bytes / (1024 * 1024)
print(f"Total size of the model pruned: {total_size_megabytes:.2f} MB")
pruned_model_path = '/content/drive/MyDrive/Tuto/model_pruned.pth'
torch.save(model.state_dict(), pruned_model_path)
print(f"Pruned model saved at: {pruned_model_path}")
pruned_model_path = 'model_pruned.pth'
torch.save(model.state_dict(), pruned_model_path)

Total size of the model pruned: 0.24 MB
Pruned model saved at: /content/drive/MyDrive/Tuto/model_pruned.pth


In [ ]:
import torch
import time
# Load the model
model_path1= '/content/drive/MyDrive/Tuto/model.pth'
state_dict1 = torch.load(model_path1)
model_path2= '/content/drive/MyDrive/Tuto/model_pruned.pth'
state_dict2 = torch.load(model_path2)
# Remove the additional keys related to pruning
state_dict2 = {key: value for key, value in state_dict2.items() if not key.endswith(('weight_orig', 'weight_mask'))}
#Test model
model1=LeNet()
model1.load_state_dict(state_dict1)
model1.eval()
start_time = time.time()
with torch.no_grad():
    for batch_idx, (data, targets) in enumerate(test_loader):
        outputs = model1(data)
        loss = criterion(outputs, targets)

        if (batch_idx + 1) % 100 == 0:
           print(f'Model 1 Loss: {loss.item():.4f}')
end_time = time.time()
elapsed_time1 = end_time - start_time
print(f"Elapsed time: {elapsed_time1} seconds")

model2=LeNet()
model2.load_state_dict(state_dict2)
model2.eval()
start_time = time.time()
with torch.no_grad():
    for batch_idx, (data, targets) in enumerate(test_loader):
        outputs = model2(data)
        loss = criterion(outputs, targets)

        if (batch_idx + 1) % 100 == 0:
           print(f'Model 2 Loss: {loss.item():.4f}')
end_time = time.time()
elapsed_time2 = end_time - start_time
print(f"Elapsed time: {elapsed_time2} seconds")

Model 1 Loss: 0.6309
Elapsed time: 5.255553483963013 seconds
Model 2 Loss: 2.3238
Elapsed time: 3.9305484294891357 seconds


In [ ]:
total_size_bytes = sum(p.numel() * p.element_size() for p in model1.parameters())
total_size_megabytes = total_size_bytes / (1024 * 1024)

print(f"Total size of the model1: {total_size_megabytes:.2f} MB")
total_size_bytes = sum(p.numel() * p.element_size() for p in model2.parameters())
total_size_megabytes = total_size_bytes / (1024 * 1024)

print(f"Total size of the model2: {total_size_megabytes:.2f} MB")

Total size of the model1: 0.24 MB
Total size of the model2: 0.24 MB


In [ ]:
#Quantization
from torch.quantization import quantize_dynamic, QuantStub, DeQuantStub
quantized_model = quantize_dynamic(model1, {torch.nn.Linear}, dtype=torch.qint8)
torch.save(quantized_model.state_dict(), 'quantized_model.pth')
quantized_model.eval()
model1.eval()
l=0
lq=0
numtry=10
for i in range(numtry):
  with torch.no_grad():
      for batch_idx, (data, targets) in enumerate(test_loader):
          outputsq = quantized_model(data)
          outputs = model1(data)
          lossq = criterion(outputsq, targets)
          loss = criterion(outputs, targets)
          if (batch_idx + 1) % 100 == 0:
            #print(f'Model Loss: {loss.item():.4f} and Model quantized Loss: {lossq.item():.4f}')
            lq+=lossq.item()
            l+=loss.item()
l=l/numtry
lq=lq/numtry
print(f'Model Loss: {l:.4f} and Model quantized Loss: {lq:.4f}')

Model Loss: 0.7011 and Model quantized Loss: 0.6988


In [ ]:
#save model
quantized_model = quantize_dynamic(model2, {torch.nn.Linear}, dtype=torch.qint8)
torch.save(quantized_model.state_dict(), 'quantized_purned_model.pth')
torch.save(quantized_model.state_dict(), 'quantized_model.pth')
torch.save(model1.state_dict(), 'model.pth')

# Quantization Model


In [ ]:
!pip install onnx
!pip install onnx2pytorch
!pip install onnx-torch
!pip install torch-summary
!pip install onnxruntime
!pip install opencv-python
!pip install pillow
!pip install onnx
!pip install onnxscript

In [ ]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

model_fp32 = '/content/drive/MyDrive/Tuto/inceptdropfaith.onnx'
model_quant = 'model.quant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant)


In [ ]:
import cv2
import torch
import onnx
import onnxruntime
import torchvision.transforms as transforms
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow


# Define the image preprocessing transforms
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the list of class names
class_names = ['D', 'P']

# Load the ONNX model
model_path1 = "/content/drive/MyDrive/Tuto/inceptdropfaith.onnx"
model_path2 = "/content/model.quant.onnx"
model1 = onnx.load(model_path1)
model2 = onnx.load(model_path2)
# Create a PyTorch model from the ONNX model
pytorch_model1 = onnxruntime.InferenceSession(model_path1)
pytorch_model2 = onnxruntime.InferenceSession(model_path2)

input_name = pytorch_model.get_inputs()[0].name
output_name = pytorch_model.get_outputs()[0].name

# Load the image
# Specify the path to the image
image_path = '/content/drive/MyDrive/Tuto/Sans titre.jpeg'

# Read the image using OpenCV
img = cv2.imread(image_path)
# Convert the image to tensor
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# Convert the OpenCV image to a PIL image
img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
with torch.no_grad():
    frame_tensor = preprocess(img).unsqueeze(0)  # Add channel dimension
    frame_tensor = frame_tensor.cpu()  # Move to CPU if the model is on GPU

    # Perform inference
    output1 = pytorch_model1.run([], {input_name: frame_tensor.numpy()})[0]
    output2 = pytorch_model2.run([], {input_name: frame_tensor.numpy()})[0]
    # Get the predicted class and accuracy
    pred_idx1 = output1.argmax()
    pred_class1 = class_names[pred_idx1]
    accuracy1 = max(output[0])
    pred_idx2 = output2.argmax()
    pred_class2 = class_names[pred_idx2]
    accuracy2 = max(output[0])

    # Print the results to console
    print(f"class={pred_class1}, accuracy={accuracy1:.2f}")
    print(f"class={pred_class2}, accuracy={accuracy2:.2f}")